# Run Main

In [32]:
import os
filepath='C:\\Users\\597667\\scitext-explorer' 
os.chdir(filepath)
%run main.py

doc
{'directory': 'C:\\Users\\597667\\Documents\\Sample', 'files': ['idaho.txt', 'billrights.txt', 'illumina.txt', 'snurgle.txt', 'foodstuf.txt']}



alg config:

 {'latent_semantic_analysis': True, 'word_frequency_table': True, 'bag_of_words': True, 'tf_idf': True}
Concept 0:
idaho
illuminati
shall
amendment
state
weishaupt
people
evidence
states
used
 
Concept 1:
used
agent
sodium
acid
antioxidant
preservative
stabilizer
shall
naturally
amendment
 
Concept 2:
idaho
shall
amendment
state
states
property
right
people
evidence
public
 
Concept 3:
illuminati
weishaupt
knigge
freemasonry
illuminism
masonic
order
bavaria
bavarian
illuminatus
 
Concept 4:
idaho
evidence
state
cartographers
maps
fact
ask
used
states
information
 
['000', '05', '08', '10', '1181', '12', '13', '1395', '14', '16', '17', '1748', '1752', '1753', '1766', '1772', '1773', '1776', '1777', '1780', '1781', '1784', '1785', '1786', '1787', '1788', '1790', '1796', '18', '1800', '1811', '18th', '19', '1901', '1932', '1950s

# Main

In [33]:
# %load main.py
#!/usr/bin/env python3
"""
Created on Sun Jun 24 18:29:24 2018

@author: kenipatel
"""

import logging
from structures import Corpus, DotPDF, DotTXT, DotCSV
from preprocess import Preprocessor
from algorithms import Algorithm, BagOfWords

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)s %(levelname)s %(message)s',
                    filename='scitext.log',
                    filemode='w')
'''
TODO:
# for future of UI: 
# Config class: init with path to folder, use those defaults
# have method that lets you save as a folder of yaml files
# Algs class have objs in list and then depending on what config/alg.yaml says, will run it.
# if select something (ie for PP) that doesn't work for ALG or VIZ, spit out warning
'''

corpus = Corpus('./config/data/text_files.yaml', 'doc')
file_object = corpus.getObj()
print(file_object)
print(file_object.grouping)
print(file_object.config)


data = Preprocessor(file_object, './config/preprocessing.yaml').run()
alg = Algorithm(data, './config/algorithms.yaml')
alg.run()


doc
{'directory': 'C:\\Users\\597667\\Documents\\Sample', 'files': ['idaho.txt', 'billrights.txt', 'illumina.txt', 'snurgle.txt', 'foodstuf.txt']}



alg config:

 {'latent_semantic_analysis': True, 'word_frequency_table': True, 'bag_of_words': True, 'tf_idf': True}
Concept 0:
idaho
illuminati
shall
amendment
state
weishaupt
people
evidence
states
used
 
Concept 1:
used
agent
sodium
acid
antioxidant
preservative
stabilizer
shall
naturally
amendment
 
Concept 2:
idaho
shall
amendment
state
states
property
right
people
evidence
public
 
Concept 3:
illuminati
weishaupt
knigge
freemasonry
illuminism
masonic
order
bavaria
bavarian
illuminatus
 
Concept 4:
idaho
evidence
state
cartographers
maps
fact
ask
used
states
information
 
['000', '05', '08', '10', '1181', '12', '13', '1395', '14', '16', '17', '1748', '1752', '1753', '1766', '1772', '1773', '1776', '1777', '1780', '1781', '1784', '1785', '1786', '1787', '1788', '1790', '1796', '18', '1800', '1811', '18th', '19', '1901', '1932', '1950s

{'bag_of_words': {'dtm': <5x2112 sparse matrix of type '<class 'numpy.int64'>'
  	with 2354 stored elements in Compressed Sparse Row format>,
  'dtm_dense': matrix([[2, 0, 1, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 1, 1, 0],
          [0, 1, 0, ..., 0, 0, 1],
          [0, 0, 0, ..., 0, 0, 0]], dtype=int64),
  'vocabulary': {'jan': 1060,
   'easier': 619,
   'represented': 1611,
   'albeit': 139,
   'completely': 425,
   'optional': 1359,
   'bringing': 302,
   'low': 1166,
   'corn': 475,
   'peace': 1389,
   'akron': 138,
   'enlightenment': 661,
   'haven': 908,
   'shortlived': 1730,
   'limits': 1138,
   'flavorings': 767,
   'hanover': 901,
   'space': 1788,
   'vehicle': 2026,
   'pictures': 1426,
   'com': 411,
   'rites': 1640,
   'theories': 1912,
   'annatto': 162,
   'required': 1615,
   'general': 841,
   'pectin': 1392,
   'normal': 1313,
   'adept': 116,
   'chewing': 379,
   'mapmaking': 1195,
   'rude': 1658,
   'societies': 1761,
   

# Algorithms

In [ ]:
# %load algorithms.py
'''
todo:
1) add TFIDF instead of CountVectorizer to algs for LSA ::::: done
2) make generators repeatable so that I can run count vectrorizer for BOW and TFIDF for LSA  # I THINK IT WORKS NOW

'''


import inspect
import utilities
# from sklearn.feature_extraction.text import CountVectorizer

import sklearn
# Import all of the scikit learn stuff

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from sklearn.cluster import KMeans, MiniBatchKMeans
import pandas as pd
from pandas import DataFrame
import warnings
import numpy


class Algorithm(object):
    
    def __init__(self, data, config_file):
        self.data = data
        pass # because the next line doesn't actually work yet, need to build a preprocessing.yaml file
        self.config = utilities.get_config(config_file)
        print('\n\n\nalg config:\n\n', self.config)
        
    def __iter__(self):
        for item in self.data:
            yield item

    def run(self):
        result_dict = {}
        # NEED TO THINK ABOUT HOW TO DO FOR MULTIPLE DOCS/SPLIT BY SENTENCES  
        if self.config['latent_semantic_analysis']:
            # print('\n\nERROR: LSA not yet implemented\n\n')
            l = LatentSemanticAnalysis(self.data)
            l.run()
            result_dict['latent_semantic_analysis'] = l.output

        if self.config['bag_of_words']:
            b = BagOfWords(self.data)
            b.run()
            result_dict['bag_of_words'] = b.output
        
        if self.config['tf_idf']:
            t = Tf_Idf(self.data)
            t.run()
            result_dict['tf_idf'] = t.output

        if self.config['word_frequency_table']:
            w = WordFreq(self.data)
            w.run()
            result_dict['word_frequency_table'] = w.output

        print(result_dict)
        return result_dict



# Base class for Vector Space Models (Bag of Words, LSA, LDA, Word2Vec, Doc2Vec)
class VectorSpaceModels(object):
    
    def __init__(self, corpus):
        self.data = corpus
        self.vectorizer = None
        self.dtm = None
        self.dtm_dense = None
        self.vocabulary = None

        
class BagOfWords(VectorSpaceModels):
    
    def __init__(self, corpus):
        super().__init__(corpus)
        
    def run(self):   
        self.vectorizer = CountVectorizer(lowercase=True, stop_words='english')
        self.dtm = self.vectorizer.fit_transform(self.data)
        dtm_dense = self.dtm.todense()
        vocabulary = self.vectorizer.vocabulary_
        self.output = {'dtm': self.dtm,
                        'dtm_dense': dtm_dense,
                        'vocabulary': vocabulary}
        
        # inspecing the program stack to get the calling functions name so we don't have to hardcode it
        # when building our output
        
###  Word Frequency table inputting BagOFWords outputting freq table
class WordFreq(BagOfWords):
    
    def __init__(self, corpus):
        super().__init__(corpus)
        self.run()
    
    def run_word_freq(self):
        bow_series = pd.Series(self.output['vocabulary'])
        bow_data = bow_series.to_frame().reset_index()
        bow_data.columns = ['Word', 'Word Count']
        bow_max = bow_data.sort_values(by='Word Count', ascending=False)
        bow_max = bow_max.set_index('Word')
        self.output = bow_max


class LatentSemanticAnalysis(VectorSpaceModels):
    '''
    currently non-functional, need to ake this take in multiple docs for comparison.
    '''

    def __init__(self, corpus):
        super().__init__(corpus)

    def run(self):
        self.vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')
        self.dtm = self.vectorizer.fit_transform(self.data)
        lsa = TruncatedSVD(n_components=200, n_iter=100)  # , algorithm = 'arpack')
        dtm_lsa = lsa.fit_transform(self.dtm)
        terms = Normalizer(copy=False).fit_transform(dtm_lsa)
        terms = self.vectorizer.get_feature_names()
        dist = 1 - cosine_similarity(svd_matrix)
        for i, comp in enumerate(lsa.components_): 
            termsInComp = zip (terms,comp)
            sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
            print("Concept %d:" % i )
            for term in sortedTerms:
                print(term[0])
            print (" ")

        # dataframe = pd.DataFrame(lsa.components_, index=["component_1","component_2"], columns=self.vectorizer.get_feature_names())

        self.output = {'dtm': self.dtm,
                        'dtm_lsa': dtm_lsa}
                        # ,'dataframe': dataframe}}
    models = dict()
    km_dict = dict()
    max_clusters = 10

    for index in range(2,max_clusters + 1):

        km = KMeans(n_clusters = index,  init = 'k-means++', max_iter = 1000, random_state = 1423)
        km.fit(dist)

        clusters = km.labels_.tolist()

        km_dict[index] = Counter(clusters)

        print(index, Counter(clusters))

        models[index] = {'KMeans Model': km,
                         'KMeans Centroids': km.cluster_centers_.argsort()[:, ::-1],
                         'Document-Clustering': Counter(clusters),
                         'Frame': pd.DataFrame({'Cluster': clusters,
                                                'Document Name': docnames})}

In [14]:
    class Tf_Idf(VectorSpaceModels):
    
    def __init__(self, corpus):
        super().__init__(corpus)
        
    def run(self):
        vectorizer = TfidfVectorizer(stop_words='english', lowercase=True, encoding='utf-8')
        
        #Tranforms corpus into vectorized words
        self.dtm = vectorizer.fit_transform(self.data)
        
        #Prints idf'd words
        print(vectorizer.get_feature_names())
        
        #Prints doc-term matrix
        print(self.dtm)
        
        #Prints and returns Data Table of doc-term matrix
        Tf_Idf_Table = pd.DataFrame(self.dtm.toarray())
        self.output = print(Tf_Idf_Table)
        
        
#
# Base class for Topic Models (Topic Modelingm Named Entity Recognition, etc.)
class TopicModels(object):
    pass
        
        
    